In [28]:
import nltk
# nltk.download()import re 
import tweepy 
# import nltk
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tweepy import OAuthHandler 
import textblob  
from textblob import TextBlob
import datetime
import re

In [29]:
def connect():
  # Replace the xxxxx with your twitter api keys
    consumer_key = 'TapBtkew11SehgUk944vZM8li'
    consumer_secret = 'PUmWoc5eaeqbkxSYwSvMFfMxdNZw1sCWJdfQ711Y4Eg4QUleyS'
    access_token = '1449420729489952770-cZEvLMGUhh4gO2JCBGDaUrGlOycCUC'
    access_token_secret = 'kTfry8Ir06moj1pGGLbu9hpFZ3Aw8AUec7Si6BgdffXe3'
    try:
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        return api
    except:
        print("Error")
        exit(1)

In [30]:
def cleanText(text):
    text = text.lower()
    # Removes all mentions (@username) from the tweet since it is of no use to us
    text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)

    # Removes any link in the text
    text = re.sub('http://\S+|https://\S+', '', text)

    # Only considers the part of the string with char between a to z or digits and whitespace characters
    # Basically removes punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Removes stop words that have no use in sentiment analysis 
    text_tokens = word_tokenize(text)
    text = [word for word in text_tokens if not word in stopwords.words()]

    text = ' '.join(text)
    return text

In [31]:
def stem(text):
    # This function is used to stem the given sentence
    porter = PorterStemmer()
    token_words = word_tokenize(text)
    stem_sentence = []
    for word in token_words:
        stem_sentence.append(porter.stem(word))
    return " ".join(stem_sentence)

In [32]:
def sentiment(cleaned_text):
    # Returns the sentiment based on the polarity of the input TextBlob object
    if cleaned_text.sentiment.polarity > 0:
        return 'positive'
    elif cleaned_text.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [33]:
date_time_str = '2019-10-24 00:00:00.00'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')

In [34]:
def fetch_tweets(twitter_page):
    print("START: Fetching " + twitter_page)
    api = connect() # Gets the tweepy API object
    processed = []
    all_tweets = []
    tweets = api.user_timeline(screen_name=twitter_page, 
                            count=200,
                            include_rts = False,
                            # Necessary to keep full_text 
                            # otherwise only the first 140 words are extracted
                            tweet_mode = 'extended'
                            )

    all_tweets.extend(tweets)
    oldest_id = tweets[-1].id

    while True:
        

        tweets = api.user_timeline(screen_name=twitter_page, 
                            count=200,
                            include_rts = False,
                            max_id = oldest_id - 1,
                            # Necessary to keep full_text 
                            # otherwise only the first 140 words are extracted
                            tweet_mode = 'extended'
                            )

        if(len(tweets) == 0): # No More Tweets from account
            break   
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)
        # print('N of tweets downloaded till now {}'.format(len(all_tweets)))
        # print(tweets[-1].created_at.replace(tzinfo=None))
        if(tweets[-1].created_at.replace(tzinfo=None) < date_time_obj):
            break

    for tweet in all_tweets:
        date = tweet.created_at
        txt = tweet.full_text
        clean_txt = cleanText(txt) # Cleans the tweet
        stem_txt = TextBlob(stem(clean_txt)) # Stems the tweet
        sent = sentiment(stem_txt) # Gets the sentiment from the tweet
        sent_cont = stem_txt.sentiment.polarity
        processed.append((tweet.id,date,txt,clean_txt,sent,sent_cont))
    print("DONE: Fetching " + twitter_page)
    return processed


In [35]:
# Keep Tweets from FAANG

def filter_tweets(row):
    if(any(ele in row for ele in ["amazon","amzn"])):
        return "AMZN"
    elif(any(ele in row for ele in ["apple","aapl"])):
        return "AAPL"
    elif(any(ele in row for ele in ["facebook","fb","meta"])):
        return "FB"
    elif(any(ele in row for ele in ["netflix","nflx"])):
        return "NFLX"    
    elif(any(ele in row for ele in ["googl","google","alphabet"])):
        return "GOOGL"
        
    else:
        return "None"

https://fairyonice.github.io/extract-someones-tweet-using-tweepy.html

In [37]:
# accounts = ["nytimesbusiness","Benzinga" "BreakoutStocks","IBDinvestors","Stocktwits","bespokeinvest","emmetlsavage","TMFJMo","CNBC"]
accounts = ["Benzinga","BreakoutStocks","IBDinvestors","Stocktwits","bespokeinvest","emmetlsavage","TMFJMo","CNBC"]

for acc in accounts:
    try:
        tweets = fetch_tweets(acc) #Change Tweeted Account Here
        bool_tweet = []
        # Converting the list into a pandas Dataframe
        df = pd.DataFrame(tweets, columns = [ 'id','date','raw_txt','clean_tweets','sentiment',"sentiment_continuous"])
        df["BelongsTo"] = df["clean_tweets"].apply(filter_tweets)
        df.to_csv("./AccountTweets/" + acc + "_All.csv") #Change the name of the CSV file to the name of the Tweeter Account
        df[df["BelongsTo"].isin(["AMZN","AAPL","FB","NFLX","GOOGL"])].to_csv("./AccountTweets/" + acc + "_OnlyFAANG.csv")
    except:
        print("Failed Retrieving Tweets For " + acc)

START: Fetching Benzinga
DONE: Fetching Benzinga
START: Fetching BreakoutStocks
DONE: Fetching BreakoutStocks
START: Fetching IBDinvestors
DONE: Fetching IBDinvestors
START: Fetching Stocktwits
DONE: Fetching Stocktwits
START: Fetching bespokeinvest
DONE: Fetching bespokeinvest
START: Fetching emmetlsavage
DONE: Fetching emmetlsavage
START: Fetching TMFJMo
DONE: Fetching TMFJMo
START: Fetching CNBC
DONE: Fetching CNBC
